In [1]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import scipy as sp
import statsmodels.formula.api as sm
from sklearn.inspection import permutation_importance
from itertools import product

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#no dado original, havia uma coluna sem nome (com os indexes). seria difícil remover assim. Nomeei-a de "A", para priorizar 
#o index do pandas.

# Função para normalizar todas as colunas de um dataframe pelo método de valor mínimo 0 e máximo 1
def normalizar(df):
    resultado = df.copy()
    for coluna in df.columns:
        if df[coluna].max() == 0:
            continue
        else:
            valor_max = df[coluna].max()
            valor_min = df[coluna].min()
            resultado[coluna] = (df[coluna] - valor_min) / (valor_max - valor_min)
    return resultado


In [3]:
#criando o DataFrame "df_chem" (e talvez outros DataFrames)
df = pd.read_csv("Minerals_Database3.csv")  # Importação do dataset para ser utilizado como dataframe
# Remoção de colunas não necessárias para o dataframe

df = df.drop(['A'], axis=1)
df = df.drop(['Name'], axis=1)

dados_cat = df.reindex(df.columns[[0,2,4]], axis = 1) # Remoção de dados categóricos
dados_categoricos = dados_cat.astype("category") # Tratamento de dados para o tipo categórico
newlist = [x for x in range(7,135)] # Range para pegarmos dados que estão em porcentagem
porcent_df = df.reindex(df.columns[newlist], axis = 1) # Coleta dos dados de elementos em porcentagem
df_chem = df.reindex(df.columns[[1,3,5,6,135,136,137]], axis=1) # Reindexação de um dataframe contendo apenas propriedades fisico-químicas
dados_categoricos, df_chem

(     Crystal Structure Diaphaneity Optical
 0                  5.0         0.0     3.0
 1                  4.0         3.0     3.0
 2                  5.0         3.0     3.0
 3                  0.0         0.0     0.0
 4                  2.0         2.0     4.0
 ...                ...         ...     ...
 3107               0.0         0.0     0.0
 3108               0.0         0.0     0.0
 3109               0.0         0.0     0.0
 3110               0.0         0.0     0.0
 3111               0.0         0.0     0.0
 
 [3112 rows x 3 columns],
       Mohs Hardness  Specific Gravity  Refractive Index  Dispersion  \
 0              4.50             3.240             1.580         0.0   
 1              2.75             3.446             1.592         0.0   
 2              2.00             4.420             2.085         0.0   
 3              0.00             0.000             0.000         0.0   
 4              5.50             1.050             1.634         0.0   
 ...        

In [4]:
# Computação e análise descritiva e estatística dos dados

# Valores de média, desvio padrão, mínimos e máximos, Q1, mediana e Q3
print(df_chem.describe())

# Moda do dataframe
df_chem.mode

       Mohs Hardness  Specific Gravity  Refractive Index   Dispersion  \
count    3112.000000       3112.000000       3112.000000  3112.000000   
mean        0.845925          0.666800          0.323626     0.000459   
std         1.844267          1.481692          0.678769     0.006851   
min         0.000000          0.000000          0.000000     0.000000   
25%         0.000000          0.000000          0.000000     0.000000   
50%         0.000000          0.000000          0.000000     0.000000   
75%         0.000000          0.000000          0.000000     0.000000   
max         9.500000          9.593000          3.383000     0.213000   

         Molar Mass  Molar Volume  Calculated Density  
count   3112.000000   3112.000000          3112.00000  
mean     607.052968      0.110810             5.28411  
std      673.908663      0.117827             2.86038  
min        1.007940      0.011030             0.07500  
25%      239.417117      0.044832             3.15475  
50%   

<bound method DataFrame.mode of       Mohs Hardness  Specific Gravity  Refractive Index  Dispersion  \
0              4.50             3.240             1.580         0.0   
1              2.75             3.446             1.592         0.0   
2              2.00             4.420             2.085         0.0   
3              0.00             0.000             0.000         0.0   
4              5.50             1.050             1.634         0.0   
...             ...               ...               ...         ...   
3107           0.00             0.000             0.000         0.0   
3108           0.00             0.000             0.000         0.0   
3109           0.00             0.000             0.000         0.0   
3110           0.00             0.000             0.000         0.0   
3111           0.00             0.000             0.000         0.0   

       Molar Mass  Molar Volume  Calculated Density  
0      817.339002      0.123390               5.498  
1      

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

tamanho = 0.25 # Fração de dados escolhida para treino e teste
seed = 2000

i = df_chem.index
i_treino, i_teste = train_test_split(i, test_size=tamanho, random_state = seed)

df_treino = df_chem.loc[i_treino]
df_teste = df_chem.loc[i_teste]

#print(df_treino)
#print()
#print(df_teste)

In [6]:
# Modelo preditivo
# Definições de atributos e targets
nome = 'df_chem'
atributos = ["Specific Gravity", "Mohs Hardness", "Dispersion", 'Molar Mass', 'Molar Volume', 'Calculated Density']
target = ["Refractive Index"]

X_treino = df_treino.reindex(atributos, axis=1).values
y_treino = df_treino.reindex(target, axis=1).values
X_teste = df_teste.reindex(atributos, axis=1).values
y_teste = df_teste.reindex(target, axis=1).values

### Baseline

In [7]:
modelo_baseline = DummyRegressor()# Criação do Modelo fictício

modelo_baseline.fit(X_treino, y_treino) # Treino do Modelo fictício

previsao = modelo_baseline.predict(X_teste) # Previsão do modelo a partir do treino
print(previsao)

[0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355
 0.33658355 0.33658355 0.33658355 0.33658355 0.33658355 0.3365

In [8]:
y_verdadeiro = y_teste
y_previsao = modelo_baseline.predict(X_teste)
RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False) # Método da raiz quadrada do erro quadrático médio
print(f'A RMSE do modelo de baseline foi uma Indice Refrativo de {RMSE: .4f}')
#print(y_verdadeiro)

A RMSE do modelo de baseline foi uma Indice Refrativo de  0.6495


In [9]:
tamanho = 0.25 # Fração de dados escolhida para treino e teste
seed = 567

i = df_chem.index
i_treino, i_teste = train_test_split(i, test_size=tamanho, random_state = seed) #Split entre dados treino e teste

df_treino = df_chem.loc[i_treino]
df_teste = df_chem.loc[i_teste]

print(df_treino)
print()
print(df_teste)

      Mohs Hardness  Specific Gravity  Refractive Index  Dispersion  \
3065            0.0               0.0             0.000         0.0   
1670            0.0               0.0             0.000         0.0   
2698            0.0               0.0             0.000         0.0   
123             0.0               0.0             0.000         0.0   
263             6.5               0.0             1.578         0.0   
...             ...               ...               ...         ...   
2064            0.0               0.0             0.000         0.0   
2953            0.0               0.0             0.000         0.0   
1580            0.0               0.0             0.000         0.0   
2031            0.0               0.0             0.000         0.0   
2932            0.0               0.0             0.000         0.0   

       Molar Mass  Molar Volume  Calculated Density  
3065   175.009500      0.033657               4.316  
1670   523.533626      0.190498        

In [10]:
# Modelo preditivo
# Definições de atributos e targets
nome = 'df_chem'
atributos = ["Specific Gravity", "Mohs Hardness", "Dispersion", 'Molar Mass', 'Molar Volume', 'Calculated Density']
target = ["Refractive Index"]

X_treino = df_treino.reindex(atributos, axis=1).values
y_treino = df_treino.reindex(target, axis=1).values
X_teste = df_teste.reindex(atributos, axis=1).values
y_teste = df_teste.reindex(target, axis=1).values

### K-NN

In [11]:
from sklearn.neighbors import KNeighborsRegressor

# criação do modelo K-NN
modelo_knn = KNeighborsRegressor()

# treinamento do modelo
modelo_knn.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_knn.predict(X_teste)
print(previsao)

[[0.2964]
 [0.3262]
 [0.    ]
 [0.    ]
 [0.3292]
 [0.3152]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.9648]
 [0.    ]
 [1.0094]
 [0.345 ]
 [1.2182]
 [0.    ]
 [0.    ]
 [1.617 ]
 [0.6434]
 [0.    ]
 [0.    ]
 [1.6868]
 [0.    ]
 [0.3112]
 [0.9658]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.323 ]
 [0.    ]
 [1.088 ]
 [0.2996]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.413 ]
 [0.    ]
 [0.6478]
 [1.627 ]
 [0.6304]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.313 ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [2.0952]
 [0.3786]
 [0.7756]
 [0.3828]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.9334]
 [0.316 ]
 [0.294 ]
 [0.    ]
 [0.9556]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [1.6674]
 [0.    ]
 [0.323 ]
 [0.    ]
 [0.    ]
 [0.3044]
 [0.    ]
 [0.    ]
 [1.059 ]
 [1.3286]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.305 ]
 [0.3048]
 [0.    ]
 [0.    ]
 [0.    ]
 [1.8112]
 [0.    ]


In [12]:
y_verdadeiro = y_teste
y_previsao = modelo_knn.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo k-NN foi um Indice Refrativo de {RMSE}.")

O RMSE do modelo k-NN foi um Indice Refrativo de 0.502087634737501.


### Árvore de Decisões

In [13]:
from sklearn.tree import DecisionTreeRegressor

# cria o modelo
modelo_dt = DecisionTreeRegressor(random_state=1024)

# treina o modelo
modelo_dt.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_dt.predict(X_teste)
#print(previsao)

In [14]:
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_teste
y_previsao = modelo_dt.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo árvore de decisão foi de {RMSE}.")

O RMSE do modelo árvore de decisão foi de 0.4366997212382474.


### Random Forest

In [15]:
#Treinamento do modelo
from sklearn.ensemble import RandomForestRegressor

# alterando o formato do target
print(y_treino.shape, y_teste.shape)
y_treino = y_treino.ravel()
y_teste = y_teste.ravel()
print(y_treino.shape, y_teste.shape)

# cria o modelo
modelo_rf = RandomForestRegressor(random_state=1024)

# treina o modelo
modelo_rf.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_rf.predict(X_teste)
print()
print(previsao)

(2334, 1) (778, 1)
(2334,) (778,)

[0.         0.         0.         0.         1.01256    1.06112
 0.         0.         0.         1.5662     0.         1.5915
 0.         1.54966    0.         0.         1.6113     1.06097
 0.         0.         1.83839    0.         0.         1.69156
 0.         0.         0.         0.         0.         0.
 0.         1.54204    0.         1.50774    0.         1.20408
 0.         0.01524    0.         1.79715    0.         0.
 1.29931    1.00736    0.         0.92954    1.43158    1.50425
 0.         0.         0.         0.         0.         0.01865
 0.         0.         0.         0.         1.94143    0.82063
 1.54042    0.         0.         0.         0.         0.
 0.         1.42125    0.         0.         0.         1.43381
 0.         0.         0.         0.         0.         0.
 0.         1.60388    0.         0.         0.         0.
 1.35443    0.         0.         1.22192    1.48683    0.
 0.         0.         0.         0.

In [16]:
#Performance do modelo de floresta aleatória
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_teste
y_previsao = modelo_rf.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo de floresta aleatória foi de {RMSE}")

O RMSE do modelo de floresta aleatória foi de 0.3411222726222785


### Regressão Linear

In [17]:
# Modelo linear para compararmos com nossa Baseline

linear = LinearRegression() # Criação do Modelo Linear

linear.fit(X_treino, y_treino) # Treinamento do Modelo Linear

previsao_linear = linear.predict(X_teste) # Previsão do modelo linear a partir do treino

#print(previsao_linear)

In [18]:
# Comparação se foi melhor ou não

y_verdadeiro = y_teste
y_previsao = linear.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)
print(f'A RMSE do modelo linear foi uma Indice Refrativo de {RMSE}')

A RMSE do modelo linear foi uma Indice Refrativo de 0.36397897320132006


Desse modo, pode-se concluir que o melhor modelo possível para a previsão do Índice Refrativo é a Regressão Linear, uma vez que comparado a todos os outros modelos apresenta o menor RMSE.